In [12]:
import pandas as pd
import minsearch

/Users/eduacapel/PycharmProjects/Fitness-assistant/minsearch.py:10: UserWarning: Now minsearch is installable via pip: 'pip install minsearch'. Remove the downloaded file and re-install it with pip.
  warnings.warn(


In [61]:
df = pd.read_csv('data.csv')
#df.drop('id', axis=1, inplace=True)

In [25]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

zsh:1: command not found: wget


In [27]:
df.columns

Index(['exercise_name', 'type_of_activity', 'type_of_equipment', 'body_part',
       'type', 'muscle_groups_activated', 'instructions'],
      dtype='object')

In [62]:
documents = df.to_dict(orient='records')
documents

[{'id': 0,
  'exercise_name': 'Push-Ups',
  'type_of_activity': 'Strength',
  'type_of_equipment': 'Bodyweight',
  'body_part': 'Upper Body',
  'type': 'Push',
  'muscle_groups_activated': 'Pectorals, Triceps, Deltoids',
  'instructions': 'Start in a high plank position with your hands under your shoulders. Lower your body until your chest nearly touches the floor. Push back up to the starting position.'},
 {'id': 1,
  'exercise_name': 'Squats',
  'type_of_activity': 'Strength',
  'type_of_equipment': 'Bodyweight',
  'body_part': 'Lower Body',
  'type': 'Push',
  'muscle_groups_activated': 'Quadriceps, Glutes, Hamstrings',
  'instructions': 'Stand with feet shoulder-width apart. Lower your body as if sitting back into a chair, keeping your chest up. Return to standing.'},
 {'id': 2,
  'exercise_name': 'Plank',
  'type_of_activity': 'Strength/Mobility',
  'type_of_equipment': 'Bodyweight',
  'body_part': 'Core',
  'type': 'Hold',
  'muscle_groups_activated': 'Rectus Abdominis, Transvers

## Ingestion

In [63]:
index= minsearch.Index(
    text_fields = [ 'exercise_name', 'type_of_activity', 'type_of_equipment',
       'body_part', 'type', 'muscle_groups_activated', 'instructions'],
    keyword_fields=['id']
)



In [64]:
query = 'give me leg exercises for hamstring'

In [65]:
index.fit(documents)

In [35]:
index.search(query, num_results=10)

[{'exercise_name': 'Hamstring Stretch',
  'type_of_activity': 'Mobility',
  'type_of_equipment': 'Bodyweight',
  'body_part': 'Lower Body',
  'type': 'Stretch',
  'muscle_groups_activated': 'Hamstrings',
  'instructions': 'Stand upright and place one heel on a bench or step. Lean forward slightly to feel a stretch in the hamstring of the elevated leg.'},
 {'exercise_name': 'Leg Press',
  'type_of_activity': 'Strength',
  'type_of_equipment': 'Machine',
  'body_part': 'Lower Body',
  'type': 'Push',
  'muscle_groups_activated': 'Quadriceps, Glutes, Hamstrings',
  'instructions': 'Sit on a leg press machine with your feet shoulder-width apart on the platform. Push the platform away by extending your legs.'},
 {'exercise_name': 'Leg Curl',
  'type_of_activity': 'Strength',
  'type_of_equipment': 'Machine',
  'body_part': 'Lower Body',
  'type': 'Pull',
  'muscle_groups_activated': 'Hamstrings',
  'instructions': 'Lie face down on a leg curl machine and curl the pad towards your glutes, th

In [47]:
import os
from dotenv import load_dotenv

load_dotenv()

# Get the API key
api_key = os.getenv('OPENAI_API_KEY')
print(f"API Key found: {api_key is not None}")


API Key found: True


## Rag flow

In [49]:
from openai import OpenAI

client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=api_key,
)

response = client.chat.completions.create(
  extra_body={},
  model="deepseek/deepseek-chat-v3.1:free",
  messages=[
    {
      "role": "user",
      "content": query
    }
  ]
)
print(response.choices[0].message.content)

Of course! Here are excellent leg exercises specifically targeting your hamstrings, categorized by difficulty and equipment needed.

### Understanding Your Hamstrings
First, it's helpful to know that the hamstrings are a group of three muscles on the back of your thigh:
*   **Semitendinosus**
*   **Semimembranosus**
*   **Biceps Femoris**

Their main jobs are to **bend the knee** and **extend the hip** (pulling your leg backward). The best exercises combine both movements.

---

### Bodyweight Exercises (Great for Beginners & Home Workouts)

#### 1. Glute Bridge
*   **Why it works:** Excellent for hip extension, which heavily involves the hamstrings and glutes.
*   **How to do it:**
    1.  Lie on your back with knees bent, feet flat on the floor about hip-width apart. Arms by your sides.
    2.  Squeeze your glutes and hamstrings to lift your hips off the ground until your body forms a straight line from shoulders to knees.
    3.  Hold for a second at the top, then slowly lower back 

In [50]:
def search(query):
    boost = {}

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )

    return results

In [52]:
documents[0]

{'exercise_name': 'Push-Ups',
 'type_of_activity': 'Strength',
 'type_of_equipment': 'Bodyweight',
 'body_part': 'Upper Body',
 'type': 'Push',
 'muscle_groups_activated': 'Pectorals, Triceps, Deltoids',
 'instructions': 'Start in a high plank position with your hands under your shoulders. Lower your body until your chest nearly touches the floor. Push back up to the starting position.'}

In [66]:
prompt_template = """
You're a figness instuctor. Answer the QUESTION based on the CONTEXT from our exercises database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

entry_template = """
exercise_name: {exercise_name}
type_of_activity: {type_of_activity}
type_of_equipment: {type_of_equipment}
body_part ; {body_part}
type : {type}
muscle_groups_activated : {muscle_groups_activated}
instructions : {instructions}
""".strip()

def build_prompt(query, search_results):

    context = ""

    for doc in search_results:
        context = context + entry_template.format(**doc) + "\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [67]:
search_results = search(query)
prompt = build_prompt(query, search_results)

In [59]:
def llm(prompt):
    response = client.chat.completions.create(
    extra_body={},
    model="deepseek/deepseek-chat-v3.1:free",
     messages=[
        {
          "role": "user",
          "content": query
        }
      ]
    )

    return response.choices[0].message.content

In [57]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [68]:
answer = rag(query)
print(answer)

Of course! Here are excellent leg exercises specifically targeting the hamstrings, categorized from beginner to advanced.

The hamstrings are actually a group of three muscles on the back of your thigh:
*   **Semitendinosus**
*   **Semimembranosus**
*   **Biceps femoris**

Their main jobs are to **bend the knee** and **extend the hip** (pulling your leg behind you). The best exercises incorporate both movements.

---

### **Crucial First Step: Warm-Up**
Always warm up before training. Try 5-10 minutes of light cardio (jogging, cycling) followed by dynamic stretches like leg swings and bodyweight squats.

---

### **Beginner Exercises**
These are great for learning the movement patterns and building a base.

#### 1. Glute Bridge
*Excellent for teaching hip extension and activating the glutes and hamstrings.*
*   **How to do it:** Lie on your back with knees bent, feet flat on the floor about hip-width apart. Drive through your heels to lift your hips off the ground until your body forms